### County Business Patterns API Tutorial
The goal in creating this notebook is to give data users who are interested in data related to the City of Milwaukee, the Milwaukee-Waukesha, WI MSA, the Milwaukee-Racine-Waukesha, WI CSA and Milwaukee County a guide to County Business Patterns data. This notebook will show a user how to:

* Build the base URL for any Census Bureau API request
* Define arguments that can be used in a CBP API query and build an example Zipcode Business Patterns (ZBP) and County Business Patterns (CBP) query
* Create a Python `class` or separate functions that create ZBP and CBP tables with data related to Milwaukee

With these objectives defined, let's get started.

#### Building a Census Bureau API request
Any request to the Census API has a few basic elements in the base URL:

```
https://api.census.gov/data/**year**/**program name**
```

First, the base URL for any dataset is always `https://api.census.gov/data`. If you want data for a specific year, such as 2019, add that to the year argument. Finally, if you want CBP data for any year from 1986 to 2019, specify the `cbp` in the program name argument. For ZBP, specify `cbp` in the program name for any data from 2017 onwards and specify `zbp` for any data from 1994 to 2018 (yes, there is some overlap in years, 2017 & 2018, between the sources). 

So, if you wanted ZBP data for the Milwaukee zip code 53201 for 2019, or CBP data for any Milwaukee-related data, your base URL should be:

```
https://api.census.gov/data/2019/cbp
```

If you want ZBP data for 2017 or 2018, you can specify `cbp` or `zbp` in the program name argument. Otherwise, if you want ZBP data for the Milwaukee zip code 53201 for 2015, your base URL should be:


```
https://api.census.gov/data/2015/zbp
```

#### Variables
There are a few variables that users should know about for County Business Patterns data:

| Variable Name | Description | Variable Type | Required | Additional Attributes |
| :------------ | :---------- | :------------ | :------- | :-------------------- |
| `CD` | Congressional District code/name? | Geography | No; not a predicate | None |
| `COUNTY` | County name/code? | Geography | No; not a predicate | None |
| `CSA` | Combined Statistical Area code/name? | Geography | No; not a predicate | None |
| `EMP` | Employment; number of employees for <br> specified sector-geography combination | Numeric | No | `EMP_N`: employee flag |
| `EMPSZES` | Employment size of establishments code | String | Default displayed | `EMPSZES_LABEL` or `EMPSZES_TTL`: description of <br> EMPSZES code |
| `ESTAB` | Number of establishments for specified <br> sector-geography combination | Numeric | No | `ESTAB_N`: establishment flag |
| `for` | Census API FIPS '`for`' clause | FIPS-for | predicate only | None |
| `NAME` | Geographic Area Name; additional attribute <br> of `GEO_ID`, geographic identifier code. | string | No | None |
| `in` | Census API FIPS '`in`' clause | FIPS-in | predicate only | None |
| `LFO` | Legal Form of Organization code, only <br> available at state and national level | string | No | `LFO_LABEL` or `LFO_TTL`: the label for <br> the LFO code |
| `NAICSYYYY` | YYYY NAICS code (industry code) | string | Default displayed | `NAICSYYYY_LABEL` or `NAICSYYYY_TTL` : the label associated <br> with the `NAICSYYYY` code |
| `PAYANN` | Annual payroll in 1,000s of <br> dollars | numeric | No | `PAYANN_F`: flag associated with `PAYANN` |
| `PAYQTR1` | 1st quarter payroll in 1,000s <br> of dollars | numeric | No | `PAYQTR1_F`: flag associated with `PAYQTR1` |
| `STATE` | State name/code? | geography | No; not a predicate | None |
| `YEAR` | Year associated with CBP data <br> queried with the API | numeric | No | None |
| `ZIPCODE` | Zipcode; only available for 2017-2019 for CBP | geography | No; not a predicate | None |

Additional variables available at https://api.census.gov/data/2019/cbp/variables.html. For specific years, change 2019 in the web address to a different year (1986-2018).

#### Building a full query
A few extra notes before building an example CBP query.

* You **can** iterate through a list of geographies with your API query. However, CBP queries can only contain lists of counties within the same state if you are trying to query for multiple counties.
* You **cannot** iterate through lists of LFO codes, EMPSZES codes, NAICS codes, etc. The list of these codes cannot be added to your predicate, but specific combinations can be added to your predicate. I will add code that allows users to iterate through combinations of LFO-EMPSZES-NAICS codes.

With that, let's start building code to query for Milwaukee-related CBP data.

In [1]:
import os
import json
import sys
import requests
import pandas as pd
import numpy as np

class milwaukee_cbp_api:
    def __init__(self, request_url, data = None):
        self.request_url = request_url
        self.data = data
        
    def load_api_key(path = "census_api.txt"):
        """ 
        This is meant to load the API key needed to request Census data.
        """
        try:
            with open(path, 'r') as f:
                # It's assumed our file contains a single line,
                # with our API key
                return f.read().strip()
        except FileNotFoundError:
            print("'%s' file not found" % path)
    
    def mke_county_cbp_data(year, api_key = None, save_path = None):
        """
        mke_county_cbp_data is a method that allows a user to get the County Business
        Patterns data for years 1998 to 2019 (TO-DO: add method to get CBP data for 
        1986-1997). Right now, this method returns all available CBP data with these 
        columns:
        
        - EMP: total employment for specified county-sector-employer sizes combination
        - EMPSZES: code for the size of employers; EMPSZES_LABEL gives the description
                   of each EMPSZES code. I will also provide a table of EMPSZES codes 
                   below. Please note that most of these codes are not used:
                   
        # | EMPSZES code | EMPSZES_LABEL |
        # | :----------- | :------------ |
        # | "001" | "All establishments" |
        # | "204" | "Establishments with no paid employees" |
        # | "205" | "Establishments with paid employees" |
        # | "207" | "Establishments with less than 10 employees" |
        # | "209" | "Establishments with less than 20 employees" |
        # | "210" | "Establishments with less than 5 employees" |
        # | "211" | "Establishments with less than 4 employees" |
        # | "212" | "Establishments with 1 to 4 employees" |
        # | "213" | "Establishments with 1 employee" |
        # | "214" | "Establishments with 2 employees" |
        # | "215" | "Establishments with 3 or 4 employees" |
        # | "219" | "Establishments with 0 to 4 employees" |
        # | "220" | "Establishments with 5 to 9 employees" |
        # | "221" | "Establishments with 5 or 6 employees" |
        # | "222" | "Establishments with 7 to 9 employees" |
        # | "223" | "Establishments with 10 to 14 employees" |
        # | "230" | "Establishments with 10 to 19 employees" |
        # | "231" | "Establishments with 10 to 14 employees" |
        # | "232" | "Establishments with 15 to 19 employees" |
        # | "235" | "Establishments with 20 or more employees" |
        # | "240" | "Establishments with 20 to 99 employees" |
        # | "241" | "Establishments with 20 to 49 employees" |
        # | "242" | "Establishments with 50 to 99 employees" |
        # | "243" | "Establishments with 50 employees or more" |
        # | "249" | "Establishments with 100 to 499 employees" |
        # | "250" | "Establishments with 100 or more employees" |
        # | "251" | "Establishments with 100 to 249 employees" |
        # | "252" | "Establishments with 250 to 499 employees" |
        # | "253" | "Establishments with 500 employees or more" |
        # | "254" | "Establishments with 500 to 999 employees" |
        # | "260" | "Establishments with 1,000 employees or more" |
        # | "261" | "Establishments with 1,000 to 2,499 employees" |
        # | "262" | "Establishments with 1,000 to 1,499 employees" |
        # | "263" | "Establishments with 1,500 to 2,499 employees" |
        # | "270" | "Establishments with 2,500 employees or more" |
        # | "271" | "Establishments with 2,500 to 4,999 employees" |
        # | "272" | "Establishments with 5,000 to 9,999 employees" |
        # | "273" | "Establishments with 5,000 employees or more" |
        # | "280" | "Establishments with 10,000 employees or more" |
        # | "281" | "Establishments with 10,000 to 24,999 employees" |
        # | "282" | "Establishments with 25,000 to 49,999 employees" |
        # | "283" | "Establishments with 50,000 to 99,999 employees" |
        # | "290" | "Establishments with 100,000 employees or more" |
        # | "298" | "Covered by administrative records"
        
        - ESTAB: Number of establishments
        - NAME: Geographic Area Name
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2019/cbp/variables/NAICS2017.json
                     
                     YYYY = 1997, 2002, 2007, 2012, or 2017
        - PAYANN: Annual payroll ($1,000)
        - PAYQTR1: First-quarter payroll ($1,000)
        - YEAR: Reference year for CBP data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       but you will be rate limited to 500 requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all CBP data for Milwaukee County, WI for a specified
                reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/cbp'
        # If data requested is from 2017-2019 (and most likely 2017-2021 once CBP data becomes available)
        if year in range(2017, 2020):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "EMPSZES", "EMPSZES_LABEL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case when year is in 2012-2016
        elif year in range(2012, 2017):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2012", "NAICS2012_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2008-2011
        elif year in range(2008, 2012):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2007", "NAICS2007_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2003-2007
        elif year in range(2003, 2008):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2002", "NAICS2002_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        elif year in range(1998, 2003):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "YEAR", "NAICS1997", "NAICS1997_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        else:
            print(str(year) + " not currently available for CBP API requests.")
            return None
        # Create data URL, Milwaukee County, WI FIPS codes are CTY_FIPS = 079 & ST_FIPS = 55
        data_url = f'{base_url}?get={comb_cols}&for=county:079&in=state:55{api_arg}'
        #print(data_url)
        # Get response from Census API
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        mke_county_cbp = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS2017", "EMPSZES"], ascending=True).reset_index(drop=True)
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_county_cbp.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_county_cbp.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_county_cbp
    
    def mke_waukesha_west_allis_msa_cbp(year, api_key = None, save_path = None):
        """
        mke_waukesha_west_allis_msa_cbp is a method that allows a user to get the County 
        Business Patterns data for years 1998 to 2019 (TO-DO: add method to get CBP data 
        for 1994-1997). Right now, this method returns all available CBP data with these 
        columns:
        
        - EMP: total employment for specified county-sector-employer sizes combination
        - EMPSZES: code for the size of employers; EMPSZES_LABEL gives the description
                   of each EMPSZES code. I will also provide a table of EMPSZES codes 
                   below. Please note that most of these codes are not used:
                   
        # | EMPSZES code | EMPSZES_LABEL |
        # | :----------- | :------------ |
        # | "001" | "All establishments" |
        # | "204" | "Establishments with no paid employees" |
        # | "205" | "Establishments with paid employees" |
        # | "207" | "Establishments with less than 10 employees" |
        # | "209" | "Establishments with less than 20 employees" |
        # | "210" | "Establishments with less than 5 employees" |
        # | "211" | "Establishments with less than 4 employees" |
        # | "212" | "Establishments with 1 to 4 employees" |
        # | "213" | "Establishments with 1 employee" |
        # | "214" | "Establishments with 2 employees" |
        # | "215" | "Establishments with 3 or 4 employees" |
        # | "219" | "Establishments with 0 to 4 employees" |
        # | "220" | "Establishments with 5 to 9 employees" |
        # | "221" | "Establishments with 5 or 6 employees" |
        # | "222" | "Establishments with 7 to 9 employees" |
        # | "223" | "Establishments with 10 to 14 employees" |
        # | "230" | "Establishments with 10 to 19 employees" |
        # | "231" | "Establishments with 10 to 14 employees" |
        # | "232" | "Establishments with 15 to 19 employees" |
        # | "235" | "Establishments with 20 or more employees" |
        # | "240" | "Establishments with 20 to 99 employees" |
        # | "241" | "Establishments with 20 to 49 employees" |
        # | "242" | "Establishments with 50 to 99 employees" |
        # | "243" | "Establishments with 50 employees or more" |
        # | "249" | "Establishments with 100 to 499 employees" |
        # | "250" | "Establishments with 100 or more employees" |
        # | "251" | "Establishments with 100 to 249 employees" |
        # | "252" | "Establishments with 250 to 499 employees" |
        # | "253" | "Establishments with 500 employees or more" |
        # | "254" | "Establishments with 500 to 999 employees" |
        # | "260" | "Establishments with 1,000 employees or more" |
        # | "261" | "Establishments with 1,000 to 2,499 employees" |
        # | "262" | "Establishments with 1,000 to 1,499 employees" |
        # | "263" | "Establishments with 1,500 to 2,499 employees" |
        # | "270" | "Establishments with 2,500 employees or more" |
        # | "271" | "Establishments with 2,500 to 4,999 employees" |
        # | "272" | "Establishments with 5,000 to 9,999 employees" |
        # | "273" | "Establishments with 5,000 employees or more" |
        # | "280" | "Establishments with 10,000 employees or more" |
        # | "281" | "Establishments with 10,000 to 24,999 employees" |
        # | "282" | "Establishments with 25,000 to 49,999 employees" |
        # | "283" | "Establishments with 50,000 to 99,999 employees" |
        # | "290" | "Establishments with 100,000 employees or more" |
        # | "298" | "Covered by administrative records"
        
        - ESTAB: Number of establishments
        - NAME: Geographic Area Name
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2019/cbp/variables/NAICS2017.json
                     
                     YYYY = 1997, 2002, 2007, 2012, or 2017
        - PAYANN: Annual payroll ($1,000)
        - PAYQTR1: First-quarter payroll ($1,000)
        - YEAR: Reference year for CBP data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       but you will be rate limited to 500 requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all CBP data for the Milwaukee-Waukesha-West Allis, WI
                MSA for a specified reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/cbp'
        # If data requested is from 2017-2019
        if year in range(2017, 2020):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "EMPSZES", "EMPSZES_LABEL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case when year is in 2012-2016
        elif year in range(2012, 2017):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2012", "NAICS2012_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2008-2011
        elif year in range(2008, 2012):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2007", "NAICS2007_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2003-2007
        elif year in range(2003, 2008):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2002", "NAICS2002_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        elif year in range(1998, 2003):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "YEAR", "NAICS1997", "NAICS1997_TTL", "EMPSZES", "EMPSZES_TTL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        else:
            print(str(year) + " not currently available for CBP API requests.")
            return None
        # Create data URL, Milwaukee-Waukesha-West Allis, WI MSA code is 33340
        data_url = f'{base_url}?get={comb_cols}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:33340{api_arg}'
        print(data_url)
        # Get response from Census API
        # Not getting response for 1998-2002, 2005--Sun. August 15th 11:10PM
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        mke_msa = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-1].sort_values(by=["NAICS2017", "EMPSZES"], ascending=True).reset_index(drop=True)
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_msa.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_msa.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_msa
    
    def mke_csa_cbp(year, api_key = None, save_path = None):
        """
        mke_csa_cbp is a method that allows a user to get the County Business Patterns data
        for years 2017 to 2019. Right now, this method returns all available CBP data with
        these columns:
        
        - EMP: total employment for specified county-sector-employer sizes combination
        - EMPSZES: code for the size of employers; EMPSZES_LABEL gives the description
                   of each EMPSZES code. I will also provide a table of EMPSZES codes 
                   below. Please note that most of these codes are not used:
                   
        # | EMPSZES code | EMPSZES_LABEL |
        # | :----------- | :------------ |
        # | "001" | "All establishments" |
        # | "204" | "Establishments with no paid employees" |
        # | "205" | "Establishments with paid employees" |
        # | "207" | "Establishments with less than 10 employees" |
        # | "209" | "Establishments with less than 20 employees" |
        # | "210" | "Establishments with less than 5 employees" |
        # | "211" | "Establishments with less than 4 employees" |
        # | "212" | "Establishments with 1 to 4 employees" |
        # | "213" | "Establishments with 1 employee" |
        # | "214" | "Establishments with 2 employees" |
        # | "215" | "Establishments with 3 or 4 employees" |
        # | "219" | "Establishments with 0 to 4 employees" |
        # | "220" | "Establishments with 5 to 9 employees" |
        # | "221" | "Establishments with 5 or 6 employees" |
        # | "222" | "Establishments with 7 to 9 employees" |
        # | "223" | "Establishments with 10 to 14 employees" |
        # | "230" | "Establishments with 10 to 19 employees" |
        # | "231" | "Establishments with 10 to 14 employees" |
        # | "232" | "Establishments with 15 to 19 employees" |
        # | "235" | "Establishments with 20 or more employees" |
        # | "240" | "Establishments with 20 to 99 employees" |
        # | "241" | "Establishments with 20 to 49 employees" |
        # | "242" | "Establishments with 50 to 99 employees" |
        # | "243" | "Establishments with 50 employees or more" |
        # | "249" | "Establishments with 100 to 499 employees" |
        # | "250" | "Establishments with 100 or more employees" |
        # | "251" | "Establishments with 100 to 249 employees" |
        # | "252" | "Establishments with 250 to 499 employees" |
        # | "253" | "Establishments with 500 employees or more" |
        # | "254" | "Establishments with 500 to 999 employees" |
        # | "260" | "Establishments with 1,000 employees or more" |
        # | "261" | "Establishments with 1,000 to 2,499 employees" |
        # | "262" | "Establishments with 1,000 to 1,499 employees" |
        # | "263" | "Establishments with 1,500 to 2,499 employees" |
        # | "270" | "Establishments with 2,500 employees or more" |
        # | "271" | "Establishments with 2,500 to 4,999 employees" |
        # | "272" | "Establishments with 5,000 to 9,999 employees" |
        # | "273" | "Establishments with 5,000 employees or more" |
        # | "280" | "Establishments with 10,000 employees or more" |
        # | "281" | "Establishments with 10,000 to 24,999 employees" |
        # | "282" | "Establishments with 25,000 to 49,999 employees" |
        # | "283" | "Establishments with 50,000 to 99,999 employees" |
        # | "290" | "Establishments with 100,000 employees or more" |
        # | "298" | "Covered by administrative records"
        
        - ESTAB: Number of establishments
        - NAME: Geographic Area Name
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2019/cbp/variables/NAICS2017.json
                     
                     YYYY = 1997, 2002, 2007, 2012, or 2017
        - PAYANN: Annual payroll ($1,000)
        - PAYQTR1: First-quarter payroll ($1,000)
        - YEAR: Reference year for CBP data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       but you will be rate limited to 500 requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all CBP data for the Milwaukee-Racine-Waukesha, WI Combined
                Statistical Area (CSA) for a specified reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/cbp'
        # If data requested is from 2017-2019
        if year in range(2017, 2020):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "EMPSZES", "EMPSZES_LABEL",
                       "ESTAB", "EMP", "PAYQTR1", "PAYANN"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        else:
            print(str(year) + " not currently available for CBP API requests.")
            return None
        # Create data URL, Milwaukee-Racine-Waukesha, WI CSA code is 376
        # If you would like to try to recreate this data for years prior to 2017, you can
        # write queries for Milwaukee, Racine, Ozaukee, Washington, Waukesha, Walworth,
        # and Dodge Counties (see https://www2.census.gov/programs-surveys/cbp/technical-documentation/
        # reference/csa-geography-reference/csa_county_reference17.txt for State & County FIPS used 
        # for creating this query) and sum over the aggregate totals. However, it should be noted
        # that some data for certain sectors is supressed for years prior to 2017 if it has too much
        # imputed data or has less than 3 establishments for a cell (combination of geography-NAICS-
        # LFO-EMPSZES etc.)
        data_url = f'{base_url}?get={comb_cols}&for=combined%20statistical%20area:376{api_arg}'
        print(data_url)
        # Get response from Census API
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        mke_csa = pd.DataFrame(data[1:], columns = data[0]).iloc[:, :-1].sort_values(by=["NAICS2017", "EMPSZES"], ascending=True).reset_index(drop=True)
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_csa.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_csa.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_csa
    
    def mke_zbp_data(year, api_key=None, requested_zips=None, save_path=None):
        """
        mke_zbp_data is a method that allows a user to get the County Business Patterns data
        for years 1998 to 2019 (TO-DO: add method to get CBP data for 1994-1997). Right now,
        this method returns all available CBP data with these columns:
        
        - EMP: total employment for specified county-sector-employer sizes combination
        - EMPSZES: code for the size of employers; EMPSZES_LABEL gives the description
                   of each EMPSZES code. I will also provide a table of EMPSZES codes 
                   below. Please note that most of these codes are not used:
                   
        # | EMPSZES code | EMPSZES_LABEL |
        # | :----------- | :------------ |
        # | "001" | "All establishments" |
        # | "204" | "Establishments with no paid employees" |
        # | "205" | "Establishments with paid employees" |
        # | "207" | "Establishments with less than 10 employees" |
        # | "209" | "Establishments with less than 20 employees" |
        # | "210" | "Establishments with less than 5 employees" |
        # | "211" | "Establishments with less than 4 employees" |
        # | "212" | "Establishments with 1 to 4 employees" |
        # | "213" | "Establishments with 1 employee" |
        # | "214" | "Establishments with 2 employees" |
        # | "215" | "Establishments with 3 or 4 employees" |
        # | "219" | "Establishments with 0 to 4 employees" |
        # | "220" | "Establishments with 5 to 9 employees" |
        # | "221" | "Establishments with 5 or 6 employees" |
        # | "222" | "Establishments with 7 to 9 employees" |
        # | "223" | "Establishments with 10 to 14 employees" |
        # | "230" | "Establishments with 10 to 19 employees" |
        # | "231" | "Establishments with 10 to 14 employees" |
        # | "232" | "Establishments with 15 to 19 employees" |
        # | "235" | "Establishments with 20 or more employees" |
        # | "240" | "Establishments with 20 to 99 employees" |
        # | "241" | "Establishments with 20 to 49 employees" |
        # | "242" | "Establishments with 50 to 99 employees" |
        # | "243" | "Establishments with 50 employees or more" |
        # | "249" | "Establishments with 100 to 499 employees" |
        # | "250" | "Establishments with 100 or more employees" |
        # | "251" | "Establishments with 100 to 249 employees" |
        # | "252" | "Establishments with 250 to 499 employees" |
        # | "253" | "Establishments with 500 employees or more" |
        # | "254" | "Establishments with 500 to 999 employees" |
        # | "260" | "Establishments with 1,000 employees or more" |
        # | "261" | "Establishments with 1,000 to 2,499 employees" |
        # | "262" | "Establishments with 1,000 to 1,499 employees" |
        # | "263" | "Establishments with 1,500 to 2,499 employees" |
        # | "270" | "Establishments with 2,500 employees or more" |
        # | "271" | "Establishments with 2,500 to 4,999 employees" |
        # | "272" | "Establishments with 5,000 to 9,999 employees" |
        # | "273" | "Establishments with 5,000 employees or more" |
        # | "280" | "Establishments with 10,000 employees or more" |
        # | "281" | "Establishments with 10,000 to 24,999 employees" |
        # | "282" | "Establishments with 25,000 to 49,999 employees" |
        # | "283" | "Establishments with 50,000 to 99,999 employees" |
        # | "290" | "Establishments with 100,000 employees or more" |
        # | "298" | "Covered by administrative records"
        
        - ESTAB: Number of establishments
        - NAME: Geographic Area Name
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2019/cbp/variables/NAICS2017.json
                     
                     YYYY = 1997, 2002, 2007, 2012, or 2017
        - PAYANN: Annual payroll ($1,000)
        - PAYQTR1: First-quarter payroll ($1,000)
        - YEAR: Reference year for CBP data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       but you will be rate limited to 500 requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all CBP data for the Milwaukee-Racine-Waukesha, WI Combined
                Statistical Area (CSA) for a specified reference year
        """
        # TO-DO: Work on methods
        return None

In [2]:
api_key = milwaukee_cbp_api.load_api_key()

In [3]:
milwaukee_cbp_api.mke_county_cbp_data(year=2019, api_key=api_key)

,NAME,YEAR,NAICS2017,NAICS2017_LABEL,EMPSZES,EMPSZES_LABEL,ESTAB,EMP,PAYQTR1,PAYANN
0,"Milwaukee County, Wisconsin",2019,00,Total for all sectors,001,All establishments,19846,453220,6722248,25843408
1,"Milwaukee County, Wisconsin",2019,00,Total for all sectors,210,Establishments with less than 5 employees,9275,0,0,0
2,"Milwaukee County, Wisconsin",2019,00,Total for all sectors,220,Establishments with 5 to 9 employees,3768,0,0,0
3,"Milwaukee County, Wisconsin",2019,00,Total for all sectors,230,Establishments with 10 to 19 employees,2837,0,0,0
4,"Milwaukee County, Wisconsin",2019,00,Total for all sectors,241,Establishments with 20 to 49 employees,2326,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4736,"Milwaukee County, Wisconsin",2019,813990,"Other similar organizations (except business, ...",001,All establishments,35,152,8842,44583
4737,"Milwaukee County, Wisconsin",2019,813990,"Other similar organizations (except business, ...",210,Establishments with less than 5 employees,23,0,0,0
4738,"Milwaukee County, Wisconsin",2019,813990,"Other similar organizations (except business, ...",220,Establishments with 5 to 9 employees,9,0,0,0
4739,"Milwaukee County, Wisconsin",2019,99,Industries not classified,001,All establishments,22,27,170,596


In [4]:
milwaukee_cbp_api.mke_waukesha_west_allis_msa_cbp(year=2019, api_key=api_key)

https://api.census.gov/data/2019/cbp?get=NAME,YEAR,NAICS2017,NAICS2017_LABEL,EMPSZES,EMPSZES_LABEL,ESTAB,EMP,PAYQTR1,PAYANN&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:33340&key=acd55604a7bbba12df83951c39d1ee8442276276


,NAME,YEAR,NAICS2017,NAICS2017_LABEL,EMPSZES,EMPSZES_LABEL,ESTAB,EMP,PAYQTR1,PAYANN
0,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,00,Total for all sectors,001,All establishments,38588,793084,11270417,44073653
1,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,00,Total for all sectors,210,Establishments with less than 5 employees,18523,0,0,0
2,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,00,Total for all sectors,220,Establishments with 5 to 9 employees,7171,0,0,0
3,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,00,Total for all sectors,230,Establishments with 10 to 19 employees,5414,0,0,0
4,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,00,Total for all sectors,241,Establishments with 20 to 49 employees,4472,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6365,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,813990,"Other similar organizations (except business, ...",001,All establishments,66,253,9673,48140
6366,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,813990,"Other similar organizations (except business, ...",210,Establishments with less than 5 employees,46,0,0,0
6367,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,813990,"Other similar organizations (except business, ...",220,Establishments with 5 to 9 employees,16,0,0,0
6368,"Milwaukee-Waukesha-West Allis, WI Metro Area",2019,99,Industries not classified,001,All establishments,43,33,201,844


In [6]:
milwaukee_cbp_api.mke_csa_cbp(year=2019, api_key=None)

https://api.census.gov/data/2019/cbp?get=NAME,YEAR,NAICS2017,NAICS2017_LABEL,EMPSZES,EMPSZES_LABEL,ESTAB,EMP,PAYQTR1,PAYANN&for=combined%20statistical%20area:376


,NAME,YEAR,NAICS2017,NAICS2017_LABEL,EMPSZES,EMPSZES_LABEL,ESTAB,EMP,PAYQTR1,PAYANN
0,"Milwaukee-Racine-Waukesha, WI CSA",2019,00,Total for all sectors,001,All establishments,49096,960607,13127721,51994036
1,"Milwaukee-Racine-Waukesha, WI CSA",2019,00,Total for all sectors,210,Establishments with less than 5 employees,23954,0,0,0
2,"Milwaukee-Racine-Waukesha, WI CSA",2019,00,Total for all sectors,220,Establishments with 5 to 9 employees,9229,0,0,0
3,"Milwaukee-Racine-Waukesha, WI CSA",2019,00,Total for all sectors,230,Establishments with 10 to 19 employees,6755,0,0,0
4,"Milwaukee-Racine-Waukesha, WI CSA",2019,00,Total for all sectors,241,Establishments with 20 to 49 employees,5503,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6912,"Milwaukee-Racine-Waukesha, WI CSA",2019,813990,"Other similar organizations (except business, ...",210,Establishments with less than 5 employees,63,0,0,0
6913,"Milwaukee-Racine-Waukesha, WI CSA",2019,813990,"Other similar organizations (except business, ...",220,Establishments with 5 to 9 employees,19,0,0,0
6914,"Milwaukee-Racine-Waukesha, WI CSA",2019,813990,"Other similar organizations (except business, ...",230,Establishments with 10 to 19 employees,3,0,0,0
6915,"Milwaukee-Racine-Waukesha, WI CSA",2019,99,Industries not classified,001,All establishments,52,42,232,1017
